In [ ]:
# Phase 1: Baseline QEC Experiments
# ==================================

import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
from datetime import datetime, timezone
from pathlib import Path

# Project imports
from src.calibration import DriftCollector
from src.probes import QubitSelector
from src.qec import RepetitionCode, QECExperimentRunner
from src.analysis import DriftErrorAnalyzer
from src.utils import QPUBudgetTracker, load_calibration_snapshot

# IBM Quantum imports
from qiskit_ibm_runtime import QiskitRuntimeService

print("Phase 1 imports loaded successfully")

## 1.1 Connect to IBM Quantum & Load Calibration Data

In [ ]:
# Initialize service and backend
service = QiskitRuntimeService(channel="ibm_quantum")

# Select backend - prefer ibm_sherbrooke or similar 127-qubit device
backend = service.least_busy(simulator=False, operational=True, min_num_qubits=27)
print(f"Selected backend: {backend.name}")
print(f"Number of qubits: {backend.num_qubits}")

# Initialize drift collector and fetch latest calibration
drift_collector = DriftCollector(backend)
calibration_snapshot = drift_collector.collect_calibration_snapshot()

print(f"\nCalibration snapshot collected at: {calibration_snapshot['timestamp']}")
print(f"Qubits with data: {len(calibration_snapshot['qubit_properties'])}")

## 1.2 Static Qubit Selection

Use the `static` strategy to select qubits based on daily calibration metrics:
- T1, T2 coherence times
- Readout error rates
- Single-qubit gate errors

In [ ]:
# Initialize qubit selector with static strategy
selector = QubitSelector(backend, strategy='static')

# Select qubits for different code distances
# Distance d requires 2d-1 data qubits + d-1 ancillas = 3d-2 qubits total for repetition code
qubit_selections = {}

for distance in [3, 5, 7]:
    n_qubits_needed = 2 * distance - 1  # Data qubits for repetition code
    selected = selector.select_qubits(
        n_qubits=n_qubits_needed,
        calibration_data=calibration_snapshot
    )
    qubit_selections[distance] = selected
    print(f"Distance {distance}: Selected qubits {selected['qubits']}")
    print(f"  Average T1: {selected['avg_t1']:.1f} µs")
    print(f"  Average readout error: {selected['avg_readout_error']:.4f}")
    print()

## 1.3 Baseline Repetition Code Experiments

Run repetition code experiments with varying:
- Code distances: d = 3, 5, 7
- Number of syndrome rounds: r = 1, 2, 3, 4
- Initial logical states: |0⟩L, |1⟩L, |+⟩L

In [ ]:
# Initialize budget tracker (10 min QPU time per 28-day window)
budget_tracker = QPUBudgetTracker(total_budget_seconds=600)

# Experiment parameters
distances = [3, 5, 7]
syndrome_rounds = [1, 2, 3, 4]
logical_states = ['0', '1', '+']  # |0⟩L, |1⟩L, |+⟩L
shots_per_circuit = 1000

# Estimate QPU time needed
n_circuits = len(distances) * len(syndrome_rounds) * len(logical_states)
estimated_time = n_circuits * shots_per_circuit * 1e-3  # ~1ms per shot estimate
print(f"Estimated experiments: {n_circuits} circuits")
print(f"Estimated QPU time: {estimated_time:.1f} seconds")
print(f"Budget remaining: {budget_tracker.remaining_budget():.1f} seconds")

In [ ]:
# Build repetition code circuits
from src.qec import RepetitionCode

experiment_circuits = []
experiment_metadata = []

for distance in distances:
    qubits = qubit_selections[distance]['qubits']
    rep_code = RepetitionCode(distance=distance, qubits=qubits)
    
    for n_rounds in syndrome_rounds:
        for init_state in logical_states:
            circuit = rep_code.build_circuit(
                n_syndrome_rounds=n_rounds,
                initial_state=init_state,
                measure_final=True
            )
            experiment_circuits.append(circuit)
            experiment_metadata.append({
                'distance': distance,
                'n_rounds': n_rounds,
                'initial_state': init_state,
                'qubits': qubits,
                'strategy': 'static'
            })

print(f"Built {len(experiment_circuits)} circuits")

In [ ]:
# Run experiments using QECExperimentRunner
runner = QECExperimentRunner(
    backend=backend,
    budget_tracker=budget_tracker
)

# Submit batch job
print("Submitting baseline experiments to IBM Quantum...")
job_id = runner.submit_batch(
    circuits=experiment_circuits,
    metadata=experiment_metadata,
    shots=shots_per_circuit
)
print(f"Job submitted: {job_id}")
print(f"Budget used: {budget_tracker.used_budget():.1f} seconds")

## 1.4 Collect and Analyze Results

In [ ]:
# Wait for job completion and collect results
results = runner.collect_results(job_id, timeout=3600)  # 1 hour timeout

print(f"Job status: {results['status']}")
if results['status'] == 'DONE':
    print(f"Collected {len(results['counts'])} result sets")

In [ ]:
# Analyze syndrome data and compute logical error rates
from src.qec import SyndromeDecoder

baseline_results = []

for i, (counts, metadata) in enumerate(zip(results['counts'], experiment_metadata)):
    decoder = SyndromeDecoder(distance=metadata['distance'])
    
    # Decode syndromes and compute logical error rate
    analysis = decoder.analyze_results(
        counts=counts,
        initial_state=metadata['initial_state'],
        n_rounds=metadata['n_rounds']
    )
    
    baseline_results.append({
        **metadata,
        'logical_error_rate': analysis['logical_error_rate'],
        'syndrome_error_rate': analysis['syndrome_error_rate'],
        'total_shots': sum(counts.values()),
        'timestamp': datetime.now(timezone.utc).isoformat()
    })

# Convert to DataFrame
df_baseline = pd.DataFrame(baseline_results)
print(df_baseline.head(10))

## 1.5 Baseline Error Budget Analysis

In [ ]:
# Compute summary statistics by distance
summary_by_distance = df_baseline.groupby('distance').agg({
    'logical_error_rate': ['mean', 'std', 'min', 'max'],
    'syndrome_error_rate': ['mean', 'std']
}).round(4)

print("Baseline Error Rates by Code Distance:")
print("=" * 50)
print(summary_by_distance)

In [ ]:
# Correlate with calibration metrics
analyzer = DriftErrorAnalyzer()

# Merge calibration data with results
correlation_data = analyzer.correlate_calibration_with_errors(
    calibration_snapshot=calibration_snapshot,
    qec_results=df_baseline,
    qubit_selections=qubit_selections
)

print("\nCorrelation Analysis:")
print(f"T1 vs Logical Error Rate: r = {correlation_data['t1_correlation']:.3f}")
print(f"T2 vs Logical Error Rate: r = {correlation_data['t2_correlation']:.3f}")
print(f"Readout Error vs Logical Error Rate: r = {correlation_data['readout_correlation']:.3f}")

## 1.6 Visualize Baseline Results

In [ ]:
import matplotlib.pyplot as plt
from src.analysis import plot_error_rates_by_distance, plot_error_budget_breakdown

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Logical error rate vs code distance
plot_error_rates_by_distance(df_baseline, ax=axes[0])
axes[0].set_title('Baseline Logical Error Rate vs Code Distance')

# Plot 2: Error budget breakdown
plot_error_budget_breakdown(correlation_data, ax=axes[1])
axes[1].set_title('Error Budget Breakdown')

plt.tight_layout()
plt.savefig('../data/figures/phase1_baseline_results.png', dpi=150, bbox_inches='tight')
plt.show()

## 1.7 Save Baseline Results

In [ ]:
from src.utils import save_experiment_results

# Save baseline results
save_experiment_results(
    results={
        'baseline_df': df_baseline,
        'calibration_snapshot': calibration_snapshot,
        'qubit_selections': qubit_selections,
        'correlation_data': correlation_data,
        'experiment_metadata': {
            'phase': 1,
            'strategy': 'static',
            'backend': backend.name,
            'timestamp': datetime.now(timezone.utc).isoformat()
        }
    },
    filepath='../data/experiments/phase1_baseline_results.json'
)

# Save DataFrame as parquet for efficient analysis
df_baseline.to_parquet('../data/experiments/phase1_baseline.parquet', index=False)

print("Phase 1 results saved successfully!")

## 1.8 Phase 1 Summary

### Key Findings
- **Baseline logical error rates** established for d=3,5,7 repetition codes
- **Error budget breakdown** quantifies contributions from gates, readout, and decoherence
- **Calibration correlations** identify which metrics best predict QEC performance

### Next Steps (Phase 2)
- Implement real-time probe refresh to capture intra-day drift
- Compare RT selection vs static selection under drift conditions
- Quantify how stale calibration data impacts logical error rates